# Analyzing FEMA's National Flood Insurance Program (NFIP) Data With DuckDB
Author: Mark Bauer

# Introduction

The FEMA National Flood Insurance Program (NFIP) website offers a trove of valuable information. Among its highlights is the [Flood Insurance Data and Analytics](https://nfipservices.floodsmart.gov/reports-flood-insurance-data) section, featuring data visualizations, tables, and reports. This project was inspired by these resources, particularly the Financial Losses by State and Policy, and Loss Statistics by Flood Zone Excel files.

Additionally, this analysis focuses solely on the [NFIP Redated Claims](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) dataset, and fortunately, it is available as a Parquet file. Given the size of this dataset, it was a good opportunity to utilize and learn more about [DuckDB](https://duckdb.org/).

# OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2
Dataset
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2. Retrieved from https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA Terms and Conditions.

Dataset Description:
>This dataset provides details on NFIP claims transactions. It is derived from the NFIP system of record, staged in the NFIP reporting platform and redacted to protect policy holder personally identifiable information.
>
>This dataset is not intended to be an official federal report, and should not be considered an official federal report.

About the National Flood Insurance Program:  
>Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (SFIP) as FEMA's fiduciary and fiscal agent. FEMA administers NFIP by ensuring insurance applications are processed properly; determining correct premiums; renewing, reforming, and cancelling insurance policies; transferring policies from the seller of the property to the purchaser of the property in certain circumstances; and processing insurance claims.
>
>The paid premiums of SFIPs and claims payments for damaged property are processed through the National Flood Insurance Fund (NFIF). NFIF was established by the National Flood Insurance Act of 1968 (42 U.S.C. 4001, et seq.), and is a centralized premium revenue and fee-generated fund that supports NFIP, which holds these U.S. Treasury funds.
>
>The Flood Insurance Claims Manual (https://nfipservices.floodsmart.gov/insurance-manuals) provides claims guidance to WYOs, vendors, adjusters, and examiners so that policyholders experience consistent and reliable service. The Manual provides processes for handling claims from the notice of loss to final payment. The NFIP has provided answers to Frequently Asked Questions (FAQs) to assist the public in understanding and navigating the data our program makes available: https://www.fema.gov/sites/default/files/documents/fema_nfip-data-faqs.pdf.

Source: [OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2)

In [1]:
# import libraries
import duckdb
import pandas as pd

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0
pandas: 1.5.1



In [3]:
ls data/

FimaNfipClaims.parquet


# Create Table Using DuckDB

In [4]:
# create a DuckDB database instance
con = duckdb.connect()

# create table of the claims dataset
con.execute("""
    CREATE TABLE IF NOT EXISTS claims AS
    FROM read_parquet('data/FimaNfipClaims.parquet')
""")

# sanity check
con.sql("""
    SELECT * FROM claims LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────┬───────────┐
│          id          │ agricultureStructu…  │       asOfDate       │ … │ censusBlockGroupFips │ latitude │ longitude │
│       varchar        │       boolean        │      timestamp       │   │       varchar        │  double  │  double   │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────┼───────────┤
│ e7af3d9f-b605-4653…  │ false                │ 2020-11-13 14:50:3…  │ … │ 010030114073         │     30.3 │     -87.7 │
│ bbaeaf64-c162-41bf…  │ false                │ 2020-12-11 16:25:4…  │ … │ 010030114072         │     30.3 │     -87.7 │
│ 256da746-b30b-4129…  │ false                │ 2020-03-27 12:15:4…  │ … │ 040131089022         │     33.5 │    -112.1 │
│ e3dcbb27-a2d0-4a9e…  │ false                │ 2020-03-27 12:15:4…  │ … │ 040131089022         │     33.5 │    -112.1 │
│ f77efb94-0188-4fa0…  │ false  

In [5]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,memory,main,claims,"[id, agricultureStructureIndicator, asOfDate, ...","[VARCHAR, BOOLEAN, TIMESTAMP, SMALLINT, SMALLI...",False


In [6]:
# count rows
con.sql("""
    SELECT COUNT(id) AS count_rows
    FROM claims
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    2671486 │
└────────────┘

# Examine and Describe Data Set

In [7]:
# preview data
con.sql("""
    SELECT *
    FROM claims
    LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬──────────┬───────────┐
│          id          │ agricultureStructu…  │       asOfDate       │ … │ censusBlockGroupFips │ latitude │ longitude │
│       varchar        │       boolean        │      timestamp       │   │       varchar        │  double  │  double   │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼──────────┼───────────┤
│ e7af3d9f-b605-4653…  │ false                │ 2020-11-13 14:50:3…  │ … │ 010030114073         │     30.3 │     -87.7 │
│ bbaeaf64-c162-41bf…  │ false                │ 2020-12-11 16:25:4…  │ … │ 010030114072         │     30.3 │     -87.7 │
│ 256da746-b30b-4129…  │ false                │ 2020-03-27 12:15:4…  │ … │ 040131089022         │     33.5 │    -112.1 │
│ e3dcbb27-a2d0-4a9e…  │ false                │ 2020-03-27 12:15:4…  │ … │ 040131089022         │     33.5 │    -112.1 │
│ f77efb94-0188-4fa0…  │ false  

In [8]:
# preview data
sql = """
    SELECT *
    FROM claims
    LIMIT 5
"""

con.sql(sql).df().iloc[:, :15]

,id,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade
0,e7af3d9f-b605-4653-958f-bc48f413766c,False,2020-11-13 14:50:38.288,2,1,NaN,2020-09-16,False,None,6.0,10.0,AE,False,NaN,0.0
1,bbaeaf64-c162-41bf-a399-862d0e832447,False,2020-12-11 16:25:40.587,0,1,NaN,2020-09-16,True,None,4.0,10.0,AE,False,7.0,4.4
2,256da746-b30b-4129-a391-93f50a6190c8,False,2020-03-27 12:15:45.887,0,1,NaN,2016-08-02,False,2,NaN,NaN,AH,False,3.0,NaN
3,e3dcbb27-a2d0-4a9e-a70d-ff7e578d4e6a,False,2020-03-27 12:15:45.887,0,1,NaN,2014-09-08,False,2,NaN,NaN,AH,False,3.0,NaN
4,f77efb94-0188-4fa0-b52e-3aa71499baeb,False,2020-03-26 12:56:27.476,0,1,NaN,2018-01-09,True,None,NaN,NaN,AE,False,NaN,NaN


In [9]:
con.sql(sql).df().iloc[:, 15:30]

,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage
0,16.0,2,False,NaN,2,1975-01-01,2008-10-20,2695.44,0.0,0.0,True,8,False,7400,0
1,14.4,3,False,54.0,1,1991-09-17,2017-10-05,2750.18,121.0,0.0,True,8,False,10800,700
2,NaN,1,False,NaN,1,1949-01-01,2014-09-02,NaN,NaN,NaN,False,B,False,153500,15800
3,NaN,1,False,NaN,1,1949-01-01,2014-09-02,25156.27,0.0,0.0,False,B,False,139500,15000
4,NaN,1,False,10.0,1,1960-01-01,2001-08-05,NaN,NaN,NaN,False,1,False,250000,0


In [10]:
con.sql(sql).df().iloc[:, 30:45]

,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber
0,2020,False,3945.0,F,2695.44,167531.0,1,N,0.0,None,0.0,0.0,0,AL0520,NaN
1,2020,True,4000.0,F,2750.18,278027.0,1,N,1371.0,F,121.0,50000.0,0,AL0520,NaN
2,2016,True,1698.0,5,0.00,140422.0,4,N,3690.0,5,0.0,21500.0,0,None,NaN
3,2014,True,30098.0,5,25156.27,140919.0,4,N,NaN,5,0.0,NaN,0,None,NaN
4,2018,True,NaN,2,0.00,NaN,4,N,NaN,0,0.0,NaN,0,None,368.0


In [11]:
con.sql(sql).df().iloc[:, 45:60]

,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis
0,NaN,NaN,False,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,185457.0,0.0,A
1,NaN,NaN,False,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,314835.0,75000.0,A
2,NaN,0.0,False,None,30000,0.0,040051,0000,None,01,01,0,175528.0,0.0,R
3,NaN,0.0,False,None,30000,0.0,040051,0000,None,None,None,0,173974.0,NaN,R
4,NaN,0.0,False,Mid-Winter California Flooding,30000,0.0,060331,0000,None,None,06,0,NaN,NaN,A


In [12]:
con.sql(sql).df().iloc[:, 60:75]

,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude
0,False,0,None,1,False,AL,Currently Unavailable,36542,01003,01003011407,010030114073,30.3,-87.7
1,False,0,AE,1,False,AL,Currently Unavailable,36542,01003,01003011407,010030114072,30.3,-87.7
2,False,1,None,1,False,AZ,Currently Unavailable,85015,04013,04013108902,040131089022,33.5,-112.1
3,False,1,None,1,False,AZ,Currently Unavailable,85015,04013,04013108902,040131089022,33.5,-112.1
4,False,0,AE,1,False,CA,Currently Unavailable,93108,06083,06083001402,060830014022,34.4,-119.6


In [13]:
# count duplicates
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM claims
    GROUP BY id
    HAVING count > 1
""")

┌─────────┬───────┐
│   id    │ count │
│ varchar │ int64 │
├─────────┴───────┤
│     0 rows      │
└─────────────────┘

In [41]:
# latest record last updated
con.sql("""
    SELECT
        asOfDate,
        dateOfLoss,
        floodEvent,
        state,
        amountPaidOnBuildingClaim
    FROM claims
    ORDER BY asOfDate DESC
    LIMIT 5
""")

┌─────────────────────────┬────────────┬────────────────────────┬─────────┬───────────────────────────┐
│        asOfDate         │ dateOfLoss │       floodEvent       │  state  │ amountPaidOnBuildingClaim │
│        timestamp        │    date    │        varchar         │ varchar │          double           │
├─────────────────────────┼────────────┼────────────────────────┼─────────┼───────────────────────────┤
│ 2024-10-04 00:59:55.552 │ 2024-09-16 │ Tropical Cyclone Eight │ NC      │                       0.0 │
│ 2024-10-04 00:58:55.563 │ 2024-09-11 │ Hurricane Francine     │ LA      │                   2900.72 │
│ 2024-10-04 00:57:54.885 │ 2024-09-11 │ Hurricane Francine     │ LA      │                       0.0 │
│ 2024-10-04 00:57:54.695 │ 2024-09-26 │ Hurricane Helene       │ FL      │                    7500.0 │
│ 2024-10-04 00:56:54.47  │ 2024-09-11 │ Hurricane Francine     │ LA      │                  57178.62 │
└─────────────────────────┴────────────┴────────────────────────

In [40]:
# latest date of loss claim
con.sql("""
    SELECT
        dateOfLoss,
        asOfDate,
        floodEvent,
        state,
        amountPaidOnBuildingClaim
    FROM claims
    ORDER BY dateOfLoss DESC
    LIMIT 5
""")

┌────────────┬─────────────────────────┬────────────┬─────────┬───────────────────────────┐
│ dateOfLoss │        asOfDate         │ floodEvent │  state  │ amountPaidOnBuildingClaim │
│    date    │        timestamp        │  varchar   │ varchar │          double           │
├────────────┼─────────────────────────┼────────────┼─────────┼───────────────────────────┤
│ 2024-10-03 │ 2024-10-03 21:40:08.5   │ NULL       │ FL      │                      NULL │
│ 2024-10-03 │ 2024-10-03 22:38:35.955 │ NULL       │ FL      │                      NULL │
│ 2024-10-03 │ 2024-10-03 17:21:21.871 │ NULL       │ FL      │                      NULL │
│ 2024-10-02 │ 2024-10-03 21:40:01.559 │ NULL       │ VA      │                      NULL │
│ 2024-10-02 │ 2024-10-03 21:40:02.111 │ NULL       │ NC      │                      NULL │
└────────────┴─────────────────────────┴────────────┴─────────┴───────────────────────────┘

# Column Info

In [16]:
# examine column metadata and push to pandas df
describe_relation = con.sql("DESCRIBE claims")

con.sql("""
    SELECT column_name, column_type
    FROM describe_relation
""").show(max_rows=80)

┌────────────────────────────────────────────┬─────────────┐
│                column_name                 │ column_type │
│                  varchar                   │   varchar   │
├────────────────────────────────────────────┼─────────────┤
│ id                                         │ VARCHAR     │
│ agricultureStructureIndicator              │ BOOLEAN     │
│ asOfDate                                   │ TIMESTAMP   │
│ basementEnclosureCrawlspaceType            │ SMALLINT    │
│ policyCount                                │ SMALLINT    │
│ crsClassificationCode                      │ SMALLINT    │
│ dateOfLoss                                 │ DATE        │
│ elevatedBuildingIndicator                  │ BOOLEAN     │
│ elevationCertificateIndicator              │ VARCHAR     │
│ elevationDifference                        │ DOUBLE      │
│ baseFloodElevation                         │ DOUBLE      │
│ ratedFloodZone                             │ VARCHAR     │
│ houseWorship          

# Summary Statistics

In [17]:
# calculate summary statistics of each column
summarize_relation = con.sql("SUMMARIZE claims")

con.sql("""
    SELECT * FROM summarize_relation
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,0000094b-1d1a-4dbe-a944-97c62b225b9c,fffffe7c-7ca2-4431-b88b-8a8216ffc934,2623176,None,None,None,None,None,2671486,0.00
1,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
2,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-10-04 00:59:55.552,479449,None,None,None,None,None,2671486,0.00
3,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.195105377264796,1.061837884284721,0,1,2,2671486,69.92
4,policyCount,SMALLINT,1,1090,399,1.2961583927447122,6.742744486702107,1,1,1,2671486,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
68,countyCode,VARCHAR,01001,78030,2946,None,None,None,None,None,2671486,2.32
69,censusTract,VARCHAR,01001020100,78030961200,59577,None,None,None,None,None,2671486,5.16
70,censusBlockGroupFips,VARCHAR,010010201001,780309612002,117722,None,None,None,None,None,2671486,5.16
71,latitude,DOUBLE,-36.0,69.9,334,33.75590024438088,5.827326735346324,29.818397464179103,30.6591295427879,39.68836090221725,2671486,1.51


In [18]:
# examine each column in sections because of large number of columns
summarize_df = con.sql("""
    SELECT * FROM summarize_relation
""").df()

summarize_df.iloc[:25, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,0000094b-1d1a-4dbe-a944-97c62b225b9c,fffffe7c-7ca2-4431-b88b-8a8216ffc934,2623176,None,None,None,None,None,2671486,0.00
1,agricultureStructureIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
2,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-10-04 00:59:55.552,479449,None,None,None,None,None,2671486,0.00
3,basementEnclosureCrawlspaceType,SMALLINT,0,4,4,1.195105377264796,1.061837884284717,0,1,2,2671486,69.92
4,policyCount,SMALLINT,1,1090,399,1.2961583927447122,6.742744486702096,1,1,1,2671486,0.00
5,crsClassificationCode,SMALLINT,1,10,10,6.708258215272243,1.5263043059761658,5,7,8,2671486,80.65
6,dateOfLoss,DATE,1978-01-01,2024-10-03,16868,None,None,None,None,None,2671486,0.00
7,elevatedBuildingIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
8,elevationCertificateIndicator,VARCHAR,1,E,9,None,None,None,None,None,2671486,77.72
9,elevationDifference,DOUBLE,-9989.0,998.0,371,1.253409617633019,29.091407737414006,0.0,1.0,2.9053433068097134,2671486,72.95


In [19]:
summarize_df.iloc[25:50, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
25,postFIRMConstructionIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
26,rateMethod,VARCHAR,1,W,22,None,None,None,None,None,2671486,1.85
27,smallBusinessIndicatorBuilding,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
28,totalBuildingInsuranceCoverage,BIGINT,0,243903000,12425,172980.48504127294,1253722.2614965665,39841,100590,221894,2671486,0.00
29,totalContentsInsuranceCoverage,BIGINT,0,6000000,3064,31127.78768283439,50873.4994227074,0,11672,45991,2671486,0.00
30,yearOfLoss,SMALLINT,1978,2024,47,2002.6466337461623,12.783941740450967,1993,2005,2012,2671486,0.00
31,primaryResidenceIndicator,BOOLEAN,false,true,2,None,None,None,None,None,2671486,0.00
32,buildingDamageAmount,BIGINT,0,927700000,202772,36354.07504655704,812334.7780209979,3426,11235,40157,2671486,23.76
33,buildingDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2671486,11.77
34,netBuildingPaymentAmount,DOUBLE,-162432.16,10006145.71,1286178,24319.109697074808,59591.412887114166,0.005608249867401044,4422.192209793248,24032.68825924921,2671486,0.00


In [20]:
summarize_df.iloc[50:, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
50,netIccPaymentAmount,DOUBLE,-6450.0,60000.0,8709,353.69498594789553,3066.941960282326,0.0,0.0,0.0,2671486,0.00
51,nfipRatedCommunityNumber,VARCHAR,000000,999999,16299,None,None,None,None,None,2671486,0.00
52,nfipCommunityNumberCurrent,VARCHAR,0000,815000,12065,None,None,None,None,None,2671486,72.10
53,nfipCommunityName,VARCHAR,ABBEVILLE COUNTY *,"ZUMBRO FALLS, CITY OF",9870,None,None,None,None,None,2671486,70.76
54,nonPaymentReasonContents,VARCHAR,01,99,23,None,None,None,None,None,2671486,68.95
55,nonPaymentReasonBuilding,VARCHAR,01,99,23,None,None,None,None,None,2671486,78.02
56,numberOfUnits,INTEGER,0,99999,450,1.3967815247755893,65.4445177643471,1,1,1,2671486,0.11
57,buildingReplacementCost,BIGINT,0,9999000000,447053,8569836.741231738,235005475.35407358,0,121139,224205,2671486,23.76
58,contentsReplacementCost,BIGINT,0,20000000,8887,2809.0997141881894,46529.60714476804,0,0,0,2671486,59.75
59,replacementCostBasis,VARCHAR,A,R,2,None,None,None,None,None,2671486,8.56


In [21]:
# examine column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM summarize_relation
    WHERE null_percentage > 0
    ORDER BY null_percentage DESC
""").show(max_rows=80)

┌────────────────────────────────────────────┬─────────────────┐
│                column_name                 │ null_percentage │
│                  varchar                   │  decimal(9,2)   │
├────────────────────────────────────────────┼─────────────────┤
│ floodCharacteristicsIndicator              │           98.51 │
│ eventDesignationNumber                     │           94.34 │
│ lowestAdjacentGrade                        │           81.26 │
│ crsClassificationCode                      │           80.65 │
│ nonPaymentReasonBuilding                   │           78.02 │
│ elevationCertificateIndicator              │           77.72 │
│ lowestFloorElevation                       │           76.38 │
│ baseFloodElevation                         │           75.70 │
│ elevationDifference                        │           72.95 │
│ floodZoneCurrent                           │           72.15 │
│ nfipCommunityNumberCurrent                 │           72.10 │
│ nfipCommunityName      

## Amount Paid on Claims Summary Statistics

In [22]:
# examine summary statisitics on amount claims paid columns
con.sql("""
    SELECT *
    FROM summarize_relation
    WHERE column_name IN (
        'amountPaidOnBuildingClaim',
        'amountPaidOnContentsClaim',
        'amountPaidOnIncreasedCostOfComplianceClaim'
    )
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,amountPaidOnBuildingClaim,DOUBLE,-162432.16,10006145.71,1286108,31531.109649180944,66190.17745513255,2320.054231129523,9616.363369788318,36641.80630487559,2671486,22.78
1,amountPaidOnContentsClaim,DOUBLE,-41276.32,757048.95,472789,7096.517735698392,22183.209325532196,0.0,3.288072593138547e-05,4796.618816175281,2671486,22.78
2,amountPaidOnIncreasedCostOfComplianceClaim,DOUBLE,-6450.0,60000.0,8752,459.73038032911796,3489.7836171167182,0.0,0.0,0.0,2671486,22.78


## Building Damage, Values, and Replacement Costs Summary Statistics

In [23]:
# examine summary statisitics on building damage amounts, property values and replacement costs columns
con.sql("""
    SELECT *
    FROM summarize_relation
    WHERE column_name IN (
        'buildingDamageAmount',
        'netBuildingPaymentAmount',
        'buildingPropertyValue',
        'buildingReplacementCost'
    )
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,buildingDamageAmount,BIGINT,0,927700000,202772,36354.07504655704,812334.778021007,3443,11216,39968,2671486,23.76
1,netBuildingPaymentAmount,DOUBLE,-162432.16,10006145.71,1286178,24319.109697074855,59591.4128871142,0.003562013112071141,4414.65311983714,24070.838022888653,2671486,0.00
2,buildingPropertyValue,BIGINT,0,9950000000,420035,7148318.887278207,199917804.10704103,60749,109691,190040,2671486,23.76
3,buildingReplacementCost,BIGINT,0,9999000000,447053,8569836.741231738,235005475.35407275,0,121224,224089,2671486,23.76


## Contents Damage, Values, and Replacement Costs Summary Statistics

In [24]:
# examine summary statisitics on contents amounts, property values and replacement costs columns
con.sql("""
    SELECT *
    FROM summarize_relation
    WHERE column_name IN (
        'contentsDamageAmount',
        'netContentsPaymentAmount',
        'contentsPropertyValue',
        'contentsReplacementCost'
    )
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,contentsDamageAmount,BIGINT,0,19230507,102299,18331.886239398373,86944.02029162786,1616,5655,18501,2671486,59.72
1,netContentsPaymentAmount,DOUBLE,-41276.32,757048.95,473195,5458.730475038984,19697.04273978,0.0,0.0,2112.972572096901,2671486,0.00
2,contentsPropertyValue,BIGINT,-1,281895323,59971,31580.59183337763,488372.3577602166,0,0,19487,2671486,59.72
3,contentsReplacementCost,BIGINT,0,20000000,8887,2809.0997141881894,46529.60714476818,0,0,0,2671486,59.75


## Elevation and Water Depth Summary Statistics

In [25]:
# examine summary statisitics on elevation and water depth columns
con.sql("""
    SELECT *
    FROM summarize_relation
    WHERE column_name IN (
        'lowestAdjacentGrade',
        'lowestFloorElevation',
        'baseFloodElevation',
        'elevationDifference',    
        'waterDepth'
    )
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,elevationDifference,DOUBLE,-9989.0,998.0,371,1.253409617633019,29.091407737413974,0.0,1.0,2.8683181106396223,2671486,72.95
1,baseFloodElevation,DOUBLE,-9999.0,9998.0,10427,126.73271926107883,767.4360045962653,6.940794471675527,9.000095426982996,13.97545617947154,2671486,75.70
2,lowestAdjacentGrade,DOUBLE,-99999.9,9998.9,12638,49.69638435096304,1483.2503230525228,3.1531864524880535,6.080567335307573,10.76175337178745,2671486,81.26
3,lowestFloorElevation,DOUBLE,-9999.0,9998.9,13294,104.32951958281964,616.4135062983048,6.533837026315136,10.251643340899367,17.304951188667037,2671486,76.38
4,waterDepth,SMALLINT,-999,999,474,4.289864898924166,16.45498525910376,0,1,2,2671486,9.40


# Analysis

In [26]:
# preview column names to use for Analysis section
(con
 .sql("SELECT * FROM claims LIMIT 10")
 .df()
 .info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 73 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   id                                          10 non-null     object        
 1   agricultureStructureIndicator               10 non-null     bool          
 2   asOfDate                                    10 non-null     datetime64[ns]
 3   basementEnclosureCrawlspaceType             10 non-null     int16         
 4   policyCount                                 10 non-null     int16         
 5   crsClassificationCode                       0 non-null      float64       
 6   dateOfLoss                                  10 non-null     datetime64[ns]
 7   elevatedBuildingIndicator                   10 non-null     bool          
 8   elevationCertificateIndicator               3 non-null      object        
 9   elevationDiff

## Total Claims

In [27]:
con.sql("""
    SELECT
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
""")

┌─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    int64    │     int64     │     int64     │             int64              │
├─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ 80638352603 │   65049593608 │   14640321888 │                      948437107 │
└─────────────┴───────────────┴───────────────┴────────────────────────────────┘

## Total Claims by State

In [28]:
con.sql("""
    SELECT
        state,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY state
    ORDER BY TotalClaim DESC
    LIMIT 20
""")

┌─────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │    int64    │     int64     │     int64     │             int64              │
├─────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ LA      │ 20869805385 │   16459981751 │    4132267467 │                      277556167 │
│ TX      │ 17218491455 │   13253973375 │    3906593054 │                       57925025 │
│ FL      │ 11433783265 │    9699132485 │    1696552579 │                       38098201 │
│ NJ      │  6470109817 │    5349181327 │     870522071 │                      250406420 │
│ NY      │  5727350837 │    4955149755 │     719127080 │                       53074002 │
│ MS      │  3122839913 │    2386323813 │     675594228 │                       60921873 │
│ NC      │  2009961692 │    1698489347 │     274446907 │                       37025438 │

## Total Claims by State for Fiscal Year 2024

In [29]:
con.sql("""
    SELECT
        state, 
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    WHERE dateOfLoss between '2023-10-01' AND '2024-09-30'
    GROUP BY state
    ORDER BY TotalClaim DESC
    LIMIT 20
""")

┌─────────┬────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ TotalClaim │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │   int64    │     int64     │     int64     │             int64              │
├─────────┼────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ FL      │  179496222 │     159954749 │      19499473 │                          42000 │
│ TX      │  141641632 │     122577288 │      19064344 │                              0 │
│ LA      │   36697924 │      34205554 │       2492369 │                              0 │
│ ME      │   34229651 │      31037588 │       3182043 │                          10019 │
│ NY      │   33349928 │      30184898 │       3143712 │                          21319 │
│ CA      │   30242469 │      28002393 │       2240076 │                              0 │
│ NJ      │   23724751 │      20819633 │       2905118 │                              0 │
│ SC      

## Total Claims by County

In [30]:
con.sql("""
    SELECT
        state,
        countyCode,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY state, countyCode
    ORDER BY TotalClaim DESC
    LIMIT 20
""")

┌─────────┬────────────┬────────────┬───────────────┬───────────────┬────────────────────────────────┐
│  state  │ countyCode │ TotalClaim │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│ varchar │  varchar   │   int64    │     int64     │     int64     │             int64              │
├─────────┼────────────┼────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ TX      │ 48201      │ 8748468467 │    6794800334 │    1933648655 │                       20019478 │
│ LA      │ 22071      │ 7296273827 │    5892518413 │    1293261898 │                      110493515 │
│ LA      │ 22051      │ 3542723793 │    2633465878 │     847684749 │                       61573166 │
│ FL      │ 12071      │ 3413583297 │    3050925028 │     360290319 │                        2367950 │
│ NJ      │ 34029      │ 2607964807 │    2160867823 │     275256283 │                      171840701 │
│ TX      │ 48167      │ 2466669297 │    1896169966 │     549528562 │    

## Total Claims by Flood Event

In [31]:
con.sql("""
    SELECT
        floodEvent,
        yearOfLoss,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS ICCClaim
    FROM claims
    WHERE floodEvent NOT NULL
    GROUP BY floodEvent, yearOfLoss
    ORDER BY TotalClaim DESC
    LIMIT 20
""")

┌──────────────────────────┬────────────┬─────────────┬───────────────┬───────────────┬───────────┐
│        floodEvent        │ yearOfLoss │ TotalClaim  │ BuildingClaim │ ContentsClaim │ ICCClaim  │
│         varchar          │   int16    │    int64    │     int64     │     int64     │   int64   │
├──────────────────────────┼────────────┼─────────────┼───────────────┼───────────────┼───────────┤
│ Hurricane Katrina        │       2005 │ 16261697056 │   12659081935 │    3360020221 │ 242594900 │
│ Hurricane Harvey         │       2017 │  9055522699 │    6925370027 │    2115077279 │  15075393 │
│ Hurricane Sandy          │       2012 │  8956609943 │    7707744797 │     951637071 │ 297228075 │
│ Hurricane Ian            │       2022 │  4757384580 │    4229314739 │     525543744 │   2526097 │
│ Hurricane Ike            │       2008 │  2702511916 │    2073801567 │     577791589 │  50918760 │
│ Mid-summer severe storms │       2016 │  2533431383 │    2175218939 │     347263588 │  10948856 │


In [32]:
con.sql("""
    SELECT
        floodEvent,
        yearOfLoss,
        state,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS ICCClaim
    FROM claims
    WHERE floodEvent NOT NULL
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC
    LIMIT 20
""")

┌──────────────────────────┬────────────┬─────────┬─────────────┬───────────────┬───────────────┬───────────┐
│        floodEvent        │ yearOfLoss │  state  │ TotalClaim  │ BuildingClaim │ ContentsClaim │ ICCClaim  │
│         varchar          │   int16    │ varchar │    int64    │     int64     │     int64     │   int64   │
├──────────────────────────┼────────────┼─────────┼─────────────┼───────────────┼───────────────┼───────────┤
│ Hurricane Katrina        │       2005 │ LA      │ 13347164644 │   10441743931 │    2721451049 │ 183969663 │
│ Hurricane Harvey         │       2017 │ TX      │  9040181063 │    6912549831 │    2112555840 │  15075393 │
│ Hurricane Ian            │       2022 │ FL      │  4711517234 │    4187737200 │     521388936 │   2391097 │
│ Hurricane Sandy          │       2012 │ NJ      │  4372115685 │    3673280769 │     457640033 │ 241194883 │
│ Hurricane Sandy          │       2012 │ NY      │  4214352197 │    3702376632 │     465410811 │  46564754 │
│ Mid-summ

## Total Claims by Flood Zone
>Formerly called floodZone. NFIP Flood Zone derived from the Flood Insurance Rate Map (FIRM) used to rate the insured property. A - Special Flood with no Base Flood Elevation on FIRM; AE, A1-A30 - Special Flood with Base Flood Elevation on FIRM; A99 - Special Flood with Protection Zone; AH, AHB* - Special Flood with Shallow Ponding; AO, AOB* - Special Flood with Sheet Flow; X, B - Moderate Flood from primary water source. Pockets of areas subject to drainage problems; X, C - Minimal Flood from primary water source. Pockets of areas subject to drainage problems; D - Possible Flood; V - Velocity Flood with no Base Flood Elevation on FIRM; VE, V1-V30 - Velocity Flood with Base Flood Elevation on FIRM; AE, VE, X - New zone designations used on new maps starting January 1, 1986, in lieu of A1-A30, V1-V30, and B and C; AR - A Special Flood Hazard Area that results from the decertification of a previously accredited flood protection system that is determined to be in the process of being restored to provide base flood protection; AR Dual Zones - (AR/AE, AR/A1-A30, AR/AH, AR/AO, AR/A) Areas subject to flooding from failure of the flood protection system (Zone AR) which also overlap an existing Special Flood Hazard Area as a dual zone; *AHB, AOB, ARE, ARH, ARO, and ARA are not risk zones shown on a map, but are acceptable values for rating purposes*

In [33]:
con.sql("""
    SELECT
        ratedFloodZone as ratedFloodZone,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ ratedFloodZone │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar     │    int64    │     int64     │     int64     │             int64              │
├────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ AE             │ 35479231566 │   29574222634 │    5389301443 │                      515707490 │
│ X              │ 13436983455 │   10383896784 │    3024472374 │                       28614297 │
│ B              │  3826110386 │    2835364624 │     971908875 │                       18836887 │
│ A              │  3130755422 │    2486940415 │     606541394 │                       37273612 │
│ C              │  2853210440 │    2119312629 │     723803760 │                       10094051 │
│ A04            │  2185871995 │    1800832671 │     364399597 │                       20639726 │
│ A03            │  

## Total Claims by Occupancy Type
>Code indicating the use and occupancy type of the insured structure. Note, 2-digit codes are for Risk Rating 2.0 policies. 1=single family residence; 2 = 2 to 4 unit residential building; 3 = residential building with more than 4 units; 4 = Non-residential building; 6 = Non Residential - Business; 11 = Single-family residential building with the exception of a mobile home or a single residential unit within a multi unit building; 12 = A residential non-condo building with 2, 3, or 4 units seeking insurance on all units; 13 = A residential non-condo building with 5 or more units seeking insurance on all units; 14 = Residential mobile/manufactured home; 15 = Residential condo association seeking coverage on a building with one or more units; 16 = Single residential unit within a multi-unit building; 17 = Non-residential mobile/manufactured home; 18 = A non-residential building; 19 = a non-residential unit within a multi-unit building

In [34]:
con.sql("""
    SELECT
        occupancyType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ occupancyType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│     int16     │    int64    │     int64     │     int64     │             int64              │
├───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│             1 │ 58074889678 │   46438196346 │   10745221948 │                      891471384 │
│             4 │  6399512277 │    4292656524 │    2096028112 │                       10827641 │
│             3 │  4398976557 │    4253333229 │     138336377 │                        7306952 │
│             2 │  3733231026 │    3410379315 │     287688446 │                       35163265 │
│             6 │  3360242402 │    2512042450 │     846579743 │                        1620209 │
│            11 │  2822556575 │    2447270648 │     373666665 │                        1619262 │
│            15 │   946313442 

## Total Claims by Flood Zone and Occupancy Type

In [35]:
con.sql("""
    SELECT
        ratedFloodZone,
        occupancyType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1, 2
    ORDER BY 3 DESC
    LIMIT 20
""")

┌────────────────┬───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ ratedFloodZone │ occupancyType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar     │     int16     │    int64    │     int64     │     int64     │             int64              │
├────────────────┼───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ AE             │             1 │ 24292519550 │   19994275329 │    3805913053 │                      492331168 │
│ X              │             1 │ 10708475718 │    8230626419 │    2450440203 │                       27409096 │
│ B              │             1 │  2959370101 │    2183341811 │     758742954 │                       17285336 │
│ A              │             1 │  2377100074 │    1942533526 │     398588313 │                       35978235 │
│ AE             │            11 │  2243498692 │    1942575827 │     299391808 │        

## Total Claims by Basement Enclosure Crawlspace Type
>Basement is defined for purposes of the NFIP as any level or story which has its floor subgrade on all sides. Basement structure values are as follows: 0 - None; 1 - Finished Basement/Enclosure; 2 - Unfinished Basement/Enclosure; 3 - Crawlspace; 4 - Subgrade Crawlspace

In [36]:
con.sql("""
    SELECT
        basementEnclosureCrawlspaceType,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌─────────────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ basementEnclosureCrawlspaceType │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│              int16              │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│                            NULL │ 55736598435 │   43974709709 │   11010313677 │                      751575049 │
│                               0 │ 14118916150 │   11621906529 │    2431677559 │                       65332063 │
│                               2 │  6023112182 │    5309665486 │     621057933 │                       92388762 │
│                               1 │  3411584798 │    2953210350 │     440219829 │                       18154620 │
│                               4 │  1348141039 │    1190101534 │     137052891 

## Total Claims by Cause of Damage
>Indicates the method by which the insured's property and contents were damaged. Legal values (value : description): 0 : Other causes; 1 : Tidal water overflow; 2 : Stream, river, or lake overflow; 3 : Alluvial fan overflow; 4 : Accumulation of rainfall or snowmelt; 7 : Erosion-demolition; 8 : Erosion-removal; 9 : Earth movement, landslide, land subsidence, sinkholes, etc.; A : Closed basin lake; B : Expedited claim handling process without site inspection; C : Expedited claim handling process follow-up site inspection; D : Expedited claim handling process by Adjusting Process Pilot Program (Remote Adjustment); NOTE: Due to certain provisions of the Upton Jones Amendment to the National Flood Insurance Act, cause of loss codes 7 and 8 may be used only if the date of loss is prior to September 23, 1995. More than one cause of loss code can be selected. For example, you may select 2 (stream, river, or lake overflow) and D (remote adjustment) on a single claim, or any combination of letters/numbers as appropriate

In [37]:
con.sql("""
    SELECT
        causeOfDamage,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌───────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ causeOfDamage │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│    varchar    │    int64    │     int64     │     int64     │             int64              │
├───────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ 4             │ 32297114786 │   25920976704 │    6094556798 │                      281581284 │
│ 1             │ 21153634243 │   17340791855 │    3445277646 │                      367564742 │
│ 2             │ 18498106661 │   14798505265 │    3483557466 │                      216043929 │
│ 0             │  6152450677 │    4980477176 │    1103245478 │                       68728023 │
│ B             │  2101842424 │    1650532490 │     440000279 │                       11309655 │
│ 3             │   236698761 │     184969651 │      49490096 │                        2239014 │
│ NULL          │    56071884 

## Total Claims by Building Description
>Indicates the description of the use of the insured building. Legal values (value : description): 01 : Main House; 02 : Detached Guest House; 03 : Detached Garage; 04 : Agricultural Building; 05 : Warehouse; 06 : Pool House, Clubhouse, Recreation Building; 07 : Tool/Storage Shed; 08 : Other; 09 : Barn; 10 : Apartment Building; 11 : Apartment - Unit; 12 : Cooperative Building; 13 : Cooperative - Unit; 14 : Commercial Building; 15 : Condominium (Entire Building); 16 : Condominium - Unit; 17 : House of Worship; 18 : Manufactured (Mobile) Home; 19 : Travel Trailer; 20 : Townhouse/Rowhouse

In [38]:
con.sql("""
    SELECT
        buildingDescriptionCode,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌─────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ buildingDescriptionCode │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│          int16          │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│                       1 │ 39094112092 │   32540928483 │    6132748786 │                      420434822 │
│                    NULL │ 38230967250 │   29855329355 │    7851193461 │                      524444435 │
│                       8 │  2754729151 │    2195108224 │     556837496 │                        2783431 │
│                       5 │   199062000 │     130086791 │      68975209 │                              0 │
│                      10 │   188592994 │     187206254 │       1386741 │                              0 │
│                       2 │    586075

## Total Claims by Rental Property Indicator
>Indicates if the property is a rental property. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. NOTE: A rental property is a property from which the owner receives payment from the occupant(s), known as tenants, in return for occupying or using the property. Rental properties may be either residential or commercial.

In [39]:
con.sql("""
    SELECT
        rentalPropertyIndicator,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS TotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS BuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS ContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS IncreasedCostOfComplianceClaim
    FROM claims
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20
""")

┌─────────────────────────┬─────────────┬───────────────┬───────────────┬────────────────────────────────┐
│ rentalPropertyIndicator │ TotalClaim  │ BuildingClaim │ ContentsClaim │ IncreasedCostOfComplianceClaim │
│         boolean         │    int64    │     int64     │     int64     │             int64              │
├─────────────────────────┼─────────────┼───────────────┼───────────────┼────────────────────────────────┤
│ false                   │ 79198225176 │   63777570301 │   14474933322 │                      945721553 │
│ true                    │  1440127427 │    1272023307 │     165388566 │                        2715554 │
└─────────────────────────┴─────────────┴───────────────┴───────────────┴────────────────────────────────┘